In [ ]:
from truncatedLaplace import TruncatedLaplace
from truncatedGaussian import TruncatedGaussian
from histogramEstimator import HistogramEstimator
import numpy as np
import seaborn as sns
import plotly.express as px
import pandas as pd
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

In [ ]:
a, b = 0, 1
delta = 0.95
gamma = 0.75
laplace_parameters = [
    (0.5, {"D": 9.25, "C": 4.63, "epsilon": 2.00}),
    (0.8, {"D": 4.38, "C": 2.19, "epsilon": 1.25}),
    (1.0, {"D": 3.16, "C": 1.58, "epsilon": 1.00}),
    (2.0, {"D": 1.27, "C": 0.64, "epsilon": 0.50}),
    (5.0, {"D": 0.44, "C": 0.22, "epsilon": 0.20}),
]

gaussian_parameters = [
    (0.3, {"D": 7.06, "C": 5.40, "epsilon": 5.56}),
    (0.5, {"D": 2.42, "C": 2.03, "epsilon": 2.00}),
    (0.6, {"D": 1.62, "C": 1.49, "epsilon": 1.39}),
    (1.0, {"D": 0.54, "C": 0.70, "epsilon": 0.50}),
    (2.0, {"D": 0.13, "C": 0.23, "epsilon": 0.13}),
]

In [ ]:
scale, params = laplace_parameters[-3]
laplace = TruncatedLaplace(a=a, b=b, scale=scale)
e = HistogramEstimator(
    mechanism=laplace,
    a=a,
    b=b,
    delta=delta,
    gamma=gamma,
    **params
)

In [ ]:
scale, params = gaussian_parameters[-3]
gaussian = TruncatedGaussian(a=a, b=b, scale=scale)
e = HistogramEstimator(
    mechanism=gaussian,
    a=a,
    b=b,
    delta=delta,
    gamma=gamma,
    **params
)

In [ ]:
n_th = e.n
n_list = np.logspace(1, np.log10(n_th), 10).astype(int)

In [ ]:
df = []
for n in tqdm(n_list):
    e.n = n
    for _ in tqdm(range(10)):
        estimate = e.estimate(0, 1)
        df.append([n, estimate])
df = pd.DataFrame(df, columns=["n", "epsilon"])

In [ ]:
df["good_estimate"] = abs(df.epsilon - e.epsilon) < gamma

In [ ]:
n_valid = df.groupby("n").good_estimate.mean()
index = (n_valid > delta).argmax()

In [ ]:
n_up = n_valid.index[index]
n_down = n_valid.index[index-1]

In [ ]:
sns.lineplot(data=df, x="n", y="epsilon", label=r"Estimated $\tilde\epsilon$ averaged across 100 runs")
plt.axhspan(
    e.epsilon - gamma,
    e.epsilon + gamma,
    color="orange",
    alpha=0.3,
    label=r"Span $\epsilon \pm \gamma$",
)
plt.axhline(y=e.epsilon, color="red", linestyle="--", label=r"Actual $\epsilon$")
practical_n = (n_up + n_down) / 2
plt.axvline(x=practical_n, color="k", linestyle="--")
plt.annotate(
    "Practical n",
    xy=(practical_n, plt.ylim()[0]),
    xytext=(-35, 15),
    textcoords="offset points",
    ha="center",
    va="top",
)
plt.axvline(x=n_th, color="k", linestyle="--")
plt.annotate(
    "Theoretical n",
    xy=(n_th, plt.ylim()[0]),
    xytext=(-40, 15),
    textcoords="offset points",
    ha="center",
    va="top",
)
plt.xscale("log")
plt.title("Truncated " + e.mechanism.__class__.__name__.replace("Truncated", ""))
plt.legend()